In [1]:
class_names = ["床面", "椅子", "収納", "玄関", "浴槽", "洗面台"]

class_labels = {i: class_name for i, class_name in enumerate(class_names)}
class_ids = [i for i in range(len(class_names))]


def get_class_key(class_name):
    return class_names.index(class_name)


def get_class_name(class_key):
    return class_names[class_key]

In [2]:
import json
import os

image_dir = "dataset/Set_A_02/images"
images = [file for file in os.listdir(image_dir) if file.endswith(".jpg")]
annotation_dir = "dataset/annotations/"
annotations = [file for file in os.listdir(annotation_dir) if file.endswith(".json")]

In [27]:
from PIL import Image, ImageDraw


def draw_polygons(image_size, annotations):
    """
    指定されたアノテーションデータに基づいて、画像上にポリゴンを描画します。
    Args:
    - image_size (tuple): 画像のサイズ（幅、高さ）
    - annotations (list of dicts): 各辞書には 'all_points_x' と 'all_points_y' のリストが含まれる

    Returns:
    - PIL.Image: ポリゴンが描画された画像
    """
    # 新しい白背景の画像を生成
    image = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(image)

    # アノテーションのリストをループし、ポリゴンを描画
    for annotation in annotations:
        shape_attr = annotation["shape_attributes"]
        # X座標とY座標のリストからタプルのリスト（頂点のリスト）を作成
        points = list(zip(shape_attr["all_points_x"], shape_attr["all_points_y"]))
        # ポリゴンを描画（黒色で塗りつぶし）
        draw.polygon(points, fill="black")

        label = annotation['region_attributes']['floor_label']

        draw.text(points[0], label, fill="red")

    return image

In [28]:
annotations = [
    {
        "shape_attributes": {
            "name": "polyline",
            "all_points_x": [14, 14, 102, 105],
            "all_points_y": [117, 237, 238, 119],
        },
        "region_attributes": {
            "floor_label": "洋室6帖",
            "class": "床面",
            "description": "洋室",
        },
    },
    {
        "shape_attributes": {
            "name": "polyline",
            "all_points_x": [59, 59, 85, 85, 103, 102],
            "all_points_y": [44, 117, 116, 67, 67, 43],
        },
        "region_attributes": {
            "floor_label": "K2",
            "class": "床面",
            "description": "K2",
        },
    },
]

# 画像のサイズ
image_size = (200, 300)  # これは適宜調整してください

# 画像を生成して表示
result_image = draw_polygons(image_size, annotations)
result_image.show()  # 結果の画像を表示果の画像を表示

UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-1: ordinal not in range(256)

In [ ]:
result = {
    "objects" : [
        {
            "label": "床面",
            "polygon": [[14, 117], [14, 237], [102, 238], [105, 119]],
            "floor_label": "洋室6帖",
        },
        {
            "label": "床面",
            "polygon": [[59, 44], [59, 117], [85, 116], [85, 67], [103, 67], [102, 43]],
            "floor_label": "K2",
        },
    ]
}

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import json

class PolyLineDataset(Dataset):
    def __init__(
            self,
            image_dir: str,
            annotation_dir: str,
            transform=None,
        ) -> None:
        super().__init__()
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [file for file in os.listdir(image_dir) if file.endswith(".jpg")]
        self.annotation_dir = annotation_dir
        self.annotations = [file for file in os.listdir(annotation_dir) if file.endswith(".json")]

    def __len__(self):
        return len(self.image_paths)

    def _get_annotation_attr(self, annotation):
        pass

    def _load_annotation(self, annotation_path):
        with open(annotation_path, "r") as f:
            annotation = json.load(f)
        return annotation